# Лабораторная работа №3
## Выполнил студент группы БВТ2005 Лесных Михаил Андреевич

### Оглавление
1. [Задание 1](#Задание-№1)
2. [Задание 2](#Задание-№2)
4. [Вывод](#Вывод)

> Дополнительные модули, использованные при выполнение лабораторной

In [61]:
# Необходим при замере скорости выполнения кода
from datetime import datetime as t
# Нужен для создания словаря в алг. Бойера-Мура
from collections import defaultdict

### Задание №1
Реализовать методы поиска подстроки в строке. Добавить возможность ввода строки и подстроки с клавиатуры. Предусмотреть возможность существования пробела. Реализовать возможность выбора опции чувствительности или нечувствительности к регистру. Оценить время работы каждого алгоритма поиска и сравнить его со временем работы стандартной функции поиска, используемой в выбранном языке программирования.

#### Алгоритм Кнута-Морриса-Пратта

In [62]:
def prefix_func(line):
    p = [0 for i in range(len(line))]
    j = 0
    i = 1
    while i < len(line):
        if line[i] != line[j]:
            if j == 0:
                p[i] = 0
                i += 1
            else:
                j = p[j - 1]
        else:
            p[i] = j + 1
            j += 1
            i += 1
    return p

def kmp_search(line, subline, reg):
    if len(line) < len(subline):
        return -1
    if reg == 0:
        line = line.lower()
        subline = subline.lower()
    p = prefix_func(subline)
    j = 0
    i = 0
    while i < len(line):
        if line[i] == subline[j]:
            i += 1
            j += 1
            if j == len(subline):
                return i - len(subline)
        else:
            if j > 0:
                j = p[j - 1]
            else:
                i += 1
    return -1
t1 = t.now()
print(kmp_search("Hello world", "or", 0))
t2 = t.now()
delta = t2 - t1
print("--- {0} ms ---".format(delta.microseconds))
t1 = t.now()
print("Hello world".find("or"))
t2 = t.now()
delta = t2 - t1
print("--- {0} ms ---".format(delta.microseconds))

7
--- 0 ms ---
7
--- 0 ms ---


#### Упрощенный алгоритм Бойера-Мура

In [63]:
def bm_search(line, subline, reg):
    if len(line) < len(subline):
        return -1
    if reg == 0:
        line = line.lower()
        subline = subline.lower()
    d = defaultdict()
    s = set()
    N = len(line)
    M = len(subline)
    for i in range(M - 2, -1, -1):
        if subline[i] not in s:
            d[subline[i]] = M - i - 1
            s.add(subline[i])
    if subline[M - 1] not in s:
        d[subline[M - 1]] = M
    d['*'] = M
    i = M - 1
    while i < N:
        k = 0
        for j in range(M - 1, -1, -1):
            if line[i - k] != subline[j]:
                if j == M - 1:
                    offset = d[line[i]] if d.get(line[i], False) else d['*']
                else:
                    offset = d[subline[j]]
                i += offset
                break
            k += 1
        
        if j == 0:
            return i - k + 1
    return -1


t1 = t.now()
print(bm_search("Hello world", "or", 0))
t2 = t.now()
delta = t2 - t1
print("--- {0} ms ---".format(delta.microseconds))
t1 = t.now()
print("Hello world".find("or"))
t2 = t.now()
delta = t2 - t1
print("--- {0} ms ---".format(delta.microseconds))

7
--- 0 ms ---
7
--- 0 ms ---


### Задание №2
Написать программу, определяющую, является ли данное
расположение «решаемым», то есть можно ли из него за конечное число
шагов перейти к правильному. Если это возможно, то необходимо найти хотя
бы одно решение - последовательность движений, после которой числа будут
расположены в правильном порядке.
#### Входные данные: массив чисел, представляющий собой расстановку в
Порядке «слева направо, сверху вниз». Число 0 обозначает пустое поле.
Например, массив [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0] представляет
собой «решенную» позицию элементов.
#### Выходные данные: если решения нет, то функция должна вернуть
Пустой массив []. Если решение есть, то необходимо представить решение —
для каждого шага записывается номер передвигаемого на данном шаге
элемента. 

In [64]:
from heapq import heappush, heappop
import math
import copy
def solve(input):
    sum = 0
    zero_coord = 0
  
    for i in range(len(input)):
        if input[i] == 0:
            zero_coord = i
            continue
        for j in range(i + 1, len(input)):
            if input[i] > input[j] and input[j] != 0:
                sum += 1
    if len(input) % 3 != 0:
        sum += zero_coord // math.sqrt(len(input)) + 1
    if sum % 2 != 0:
        return []
    
    else:
        start = chain(input)
        temp = copy.deepcopy(input)
        temp.sort()
        del temp[0]
        temp.append(0)
        end = chain(temp)
        result = a_star(start, end.last_node())
        for node in result.history:
            print(node)
        print(result)
        return 1
    
def a_star(start_chain, goal_node):
    node_hash = {}
    chains_queue = []
    heappush(chains_queue, start_chain)
    while chains_queue:
        cur_chain = heappop(chains_queue)
        cur_node = cur_chain.last_node()
        if cur_node == goal_node:
            return cur_chain
        node_hash[cur_node] = cur_chain.g()
        for chain in cur_chain.get_neighbours():
            if chain.last_node() in node_hash:
                if chain.g() >= node_hash[chain.last_node()]:
                    continue
                node_hash[chain.last_node()] = chain.g()
            heappush(chains_queue, chain)

def manh_dst_matrix(a, b, n):
    return abs(a % n - b % n) + abs(a // n - b // n)

class chain:
    def __str__(self):
        i = 0
        sstr = ""
        while i < self.size * self.size:
            sstr += str(self.board_state[i]) + " "
            if i % self.size == self.size - 1:
                sstr += "\n"
            i += 1
        return sstr
    
    def __init__(self, board_state, history=[]):
        self.board_state = list(board_state)
        self.size = int(math.sqrt(len(board_state)))
        self.history = history
        self.quad_size = int(self.size * self.size)
   
    def manh_dst(self):
        dst = 0
        for i in range(0, self.quad_size):
            dst += manh_dst_matrix((self.board_state[i] - 1) % self.quad_size, i, self.size)
        return int(dst)
    def last_move(self):
        if self.board_state[-1] == self.quad_size - 1 or self.board_state[-1] == self.quad_size - self.size:
            return 0
        return 2
    
    def h(self):
        return self.manh_dst() + self.last_move()
    
    def g(self):
        return len(self.history)
    
    def f(self):
        return self.h()
    
    def last_node(self):
        return str(self.board_state)
    def __lt__(self, other):
        return self.f() < other.f()
    
    def get_neighbours(self):
        neighs = []
        zero_coord = self.board_state.index(0)

        if zero_coord + 1 < self.size * self.size and manh_dst_matrix(zero_coord, zero_coord + 1, self.size) == 1:
            new_state = self.board_state.copy()
            new_state[zero_coord], new_state[zero_coord + 1] = new_state[zero_coord + 1], new_state[zero_coord]
            neighs.append(chain(new_state, self.history + [self]))

        if zero_coord - 1 >= 0 and manh_dst_matrix(zero_coord, zero_coord - 1, self.size) == 1:
            new_state = self.board_state.copy()
            new_state[zero_coord], new_state[zero_coord - 1] = new_state[zero_coord - 1], new_state[zero_coord]
            neighs.append(chain(new_state, self.history + [self]))

        if zero_coord + self.size < self.size * self.size and manh_dst_matrix(zero_coord, zero_coord + self.size,
                                                                       self.size) == 1:
            new_state = self.board_state.copy()
            new_state[zero_coord], new_state[zero_coord + self.size] = new_state[zero_coord + self.size], new_state[
                zero_coord]
            neighs.append(chain(new_state, self.history + [self]))

        if zero_coord - self.size >= 0 and manh_dst_matrix(zero_coord, zero_coord - self.size, self.size) == 1:
            new_state = self.board_state.copy()
            new_state[zero_coord], new_state[zero_coord - self.size] = new_state[zero_coord - self.size], new_state[
                zero_coord]
            neighs.append(chain(new_state, self.history + [self]))

        return neighs
t1 = t.now()
print(solve([6, 4, 5, 3, 7, 13, 2, 8, 1, 11, 14, 9, 10, 15, 12, 0]))
#[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0] 0:00:00.000999
#[6, 4, 5, 3, 7, 13, 2, 8, 1, 11, 14, 9, 10, 15, 12, 0] 0:00:07.205843
t2 = t.now()
delta = t2 - t1
print(delta)

6 4 5 3 
7 13 2 8 
1 11 14 9 
10 15 12 0 

6 4 5 3 
7 13 2 8 
1 11 14 9 
10 15 0 12 

6 4 5 3 
7 13 2 8 
1 11 0 9 
10 15 14 12 

6 4 5 3 
7 13 2 8 
1 0 11 9 
10 15 14 12 

6 4 5 3 
7 0 2 8 
1 13 11 9 
10 15 14 12 

6 4 5 3 
7 2 0 8 
1 13 11 9 
10 15 14 12 

6 4 5 3 
7 2 11 8 
1 13 0 9 
10 15 14 12 

6 4 5 3 
7 2 11 8 
1 13 14 9 
10 15 0 12 

6 4 5 3 
7 2 11 8 
1 13 14 9 
10 0 15 12 

6 4 5 3 
7 2 11 8 
1 0 14 9 
10 13 15 12 

6 4 5 3 
7 2 11 8 
1 14 0 9 
10 13 15 12 

6 4 5 3 
7 2 0 8 
1 14 11 9 
10 13 15 12 

6 4 0 3 
7 2 5 8 
1 14 11 9 
10 13 15 12 

6 0 4 3 
7 2 5 8 
1 14 11 9 
10 13 15 12 

6 2 4 3 
7 0 5 8 
1 14 11 9 
10 13 15 12 

6 2 4 3 
7 5 0 8 
1 14 11 9 
10 13 15 12 

6 2 4 3 
7 5 11 8 
1 14 0 9 
10 13 15 12 

6 2 4 3 
7 5 11 8 
1 14 9 0 
10 13 15 12 

6 2 4 3 
7 5 11 8 
1 14 9 12 
10 13 15 0 

6 2 4 3 
7 5 11 8 
1 14 9 12 
10 13 0 15 

6 2 4 3 
7 5 11 8 
1 14 9 12 
10 0 13 15 

6 2 4 3 
7 5 11 8 
1 0 9 12 
10 14 13 15 

6 2 4 3 
7 5 11 8 
1 9 0 12 
10 14 13 15 

6 2 4 3 
7 

### Вывод